# Analyse, classification et indexation des données: feuille 8
### Réduction de dimension - Analyse linéaire discriminante (LDA)

#### Avant de commencer

Dans cet exercice, on s'intéresse à l'application de l'analyse linéaire discriminante (LDA) à un corpus de données. Le but est de réduire la dimension tout en conservant une séparation entre les classes. 

<i>Questions préliminaires : </i> 

1. Quelle est la différence entre une ACP et une LDA ?

2. Quelle est la fonction objective que l'on cherche à minimiser quand on fait une LDA ?

## Exercice 1.

Dans cet exercice, on considère le corpus sur la classification des vins. Ce corpus fait partie de l'ensemble des <code>datasets</code> du module <code>sklearn.datasets</code>. Il peut être chargé en invoquant la méthode <code>load_wine()</code>.


### Préparation des données 

1. Chargez le corpus, explorez-le. 

In [ ]:
### CORRECTION
import pandas as pa
from sklearn.datasets import load_wine
wine = load_wine()
print(wine.DESCR)

2. Créez un <code>DataFrame data</code>  contenant les variables indépendantes et mettez les classes dans une variable dépendante $target$. Quelle est la taille du corpus ? Combien de variables comporte-t-il ?

In [ ]:
### CORRECTION
data = pa.DataFrame(data=wine.data, columns=wine.feature_names)
target = pa.DataFrame(data=wine.target, columns=['class'])
print(data.shape)
print(target.shape)

In [ ]:
### CORRECTION
print(wine.feature_names)
print(wine.target_names)

3. En utilisant la fonction <code>concat</code>, constituez un seul corpus <code>dataset</code> contenant et <code>data</code> et <code>target</code>. Faites un mélange des données 

In [ ]:
### CORRECTION
dataset = pa.concat([data, target], axis=1)
dataset = dataset.sample(frac=1)
dataset.head()

In [ ]:
### CORRECTION
dataset.info()

#### LDA "à la main"

Dans un premier temps, nous allons appliquer la méthode d'analyse linéaire discriminante juste sur deux classes. Nous allons donc d'abord ne garder que les vins des classes 1 et 2. Nous allons également ne conserver que deux descripteurs <code>alcohol</code> et <code>color_intensity</code>.

1. Ecrivez les instructions permettant de faire ce filtrage

In [ ]:
### CORRECTION
dataset = dataset[['alcohol', 'color_intensity', 'class']][ dataset['class']!= 0]
dataset.shape

2. Quelles sont les étapes (théoriques) pour réaliser une LDA ?

### CORRECTION

 1. On calcule les matrices de dispersion de chacune des classes S1 et S2

 2. On calcule la matrice de dispersion intra-classes : 

$$
S_w = S_1 + S_2.
$$

$$
S_i = (n_i - 1) \times COV_i
$$
    où $n_i$ est le nombre d'éléments de classe $i$ et $COV_i$ la matrice de covariance des variables restreintes à la classe $i$.  

 3. Le vecteur de la meilleur direction est donné par :

$$
S_w^{-1} * (\mu_1 - \mu_2)
$$
    où $\mu_i$ est la moyenne des variables restreintes à la classe $i$.

3. Application numérique : appliquer les étapes de la questions précédente pour réaliser une LDA. Quel est le vecteur directeur du meilleur axe de projection selon la méthode LDA ?

In [ ]:
### CORRECTION

import numpy as np
from numpy.linalg import inv

y = dataset['class']
X = dataset.drop(['class'], axis=1)

def CalculLDA(X):
    X1 = X[y==1]
    X2 = X[y==2]
    S1 = (len(X1)-1)*X1.cov()
    S2 = (len(X2)-1)*X2.cov()
    #print(S1)
    #print(S2)
    Sw = S1 + S2
    #print(Sw)
    invSw = inv(Sw)
    mu1 = X[y==1].mean()
    mu2 = X[y==2].mean()
    return np.matmul(invSw, mu1 - mu2)
    
CalculLDA(X)

3. Visualiser le résultat. Donnez les deux graphiques, avant et après la projection.

In [ ]:
### CORRECTION
v = CalculLDA(X)
X_proj = np.dot(X, v) #équivalent à X_proj = np.dot(v.T, X.T)
print(X_proj)

##### Avant la projection : 

In [ ]:
### CORRECTION
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter(X['alcohol'], X['color_intensity'], c=y)
#print(Xsave)

#### Après la projection :

In [ ]:
### CORRECTION
plt.scatter(X_proj, np.zeros(X_proj.shape[0]), c=y)

### LDA avec <code>sklearn</code>

Réalisez une LDA en utilisant la bibliothèque <code>sklearn.discriminant_analysis</code>. Comparez le résultat avec  la projection que vous avez obtenu dans la section précédente. 

In [ ]:
### CORRECTION
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()

In [ ]:
### CORRECTION
wine = load_wine()
X = pa.DataFrame(data=wine.data, columns=wine.feature_names)[['alcohol','color_intensity']][wine.target != 0]
y = wine.target[wine.target != 0]
X.head()

In [ ]:
### CORRECTION
X_lda = lda.fit_transform(X, y)
lda.explained_variance_ratio_

In [ ]:
### CORRECTION
plt.scatter(X_lda[:,0], np.zeros(X_lda.shape[0]), c=y)

### LDA sur tout le corpus 

Nous allons à présent appliquer la LDA sur tout le corpus. L'objectif est d'observer l'impact de la projection à la fois sur l'efficacité (<code>accuracy</code>) de la classification et sur le temps d'apprentissage. 

1. Rechargez les données et faites votre LDA.

In [ ]:
### CORRECTION
wine = load_wine()
X = pa.DataFrame(data=wine.data, columns=wine.feature_names)
y = wine.target

In [ ]:
### CORRECTION
ldaw = LinearDiscriminantAnalysis()
X_ldaw = ldaw.fit_transform(X, y)

2. Quel est le ratio de la variance expliqué par les axes obtenus ?

In [ ]:
### CORRECTION
ldaw.explained_variance_ratio_

3. Visualiser le résultat. 

In [ ]:
### CORRECTION
plt.scatter(X_ldaw[:, 0], X_ldaw[:, 1], c=y)
plt.xlabel('LDA1')
plt.ylabel('LDA2')

### Classification avant/après réduction

1. En utilisant un classifieur bayésien MAP, comparez les résultats obtenus avec les échantillons bruts et les résultats obtenus avec les échantillons projetés sur les axes fournis par la LDA. Comparez également les temps d'entraînement.

In [ ]:
### CORRECTION
wine = load_wine()
X = pa.DataFrame(data=wine.data, columns=wine.feature_names)
y = wine.target

In [ ]:
### CORRECTION
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
### CORRECTION
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

nb = GaussianNB()
print('Sans réduction :')
%timeit nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy: ',acc)

In [ ]:
### CORRECTION
#Avec réduction de dimension :

lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)
nb = GaussianNB()
print('Après réduction :')
%timeit nb.fit(X_train_lda, y_train)
y_pred = nb.predict(X_test_lda)
acc = accuracy_score(y_test, y_pred)
print('accuracy: ',acc)

2. Même question avec un $k$-nn.

In [ ]:
### CORRECTION

from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")

print('Sans réduction :')
knn = KNeighborsClassifier()
%timeit knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy: ',acc)

#Avec réduction de dimension :
print('Après réduction :')
knn = KNeighborsClassifier()
%timeit knn.fit(X_train_lda, y_train)
y_pred = knn.predict(X_test_lda)
acc = accuracy_score(y_test, y_pred)
print('accuracy: ',acc)

### Exercice 2.

Dans cet exercice, nous allons travailler avec le même corpus que le précédent TD : les données sur le cancer du sein. Le corpus peut être chargé par l'instruction <code> load_breast_cancer</code> de la bibliothèque <code>sklearn.datasets</code>.




Comparer les effets de l'ACP et de la LDA sur la classification de ce corpus.

In [ ]:
### CORRECTION

from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data
y = data.target

1. Faites une ACP en utilisant le module <code>PCA</code> de la bibliothèque <code>sklearn.decomposition</code>.  

Attention : pensez à centrer et réduire vos données.

In [ ]:
### CORRECTION

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
z = scaler.fit_transform(X)

In [ ]:
### CORRECTION

from sklearn.decomposition import PCA
acp = PCA()

In [ ]:
### CORRECTION

coord = acp.fit_transform(z)
print(acp.n_components_)

2. Affichez l'éboulie des valeurs propres et indiquer le nombre d'axes à retenir en utilisant le critère du coude. 

In [ ]:
### CORRECTION

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.grid()
plt.plot(np.arange(1,acp.n_components_+1),acp.explained_variance_ratio_) 
plt.title("Scree plot") 
plt.ylabel("Eigen values") 
plt.xlabel("Factor number") 
plt.show()
#On applique le critère du coude et on choisit de ne retenir que les trois premiers axes. 

In [ ]:
### CORRECTION

p = 3
acp = PCA(n_components=p)
coord = acp.fit_transform(z)
print(acp.explained_variance_ratio_.sum())

3. Voyons ce qu'est l'impact de l'ACP

In [ ]:
### CORRECTION
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y)
nb = GaussianNB()
print('Sans réduction :')
%timeit nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy: ',acc)

In [ ]:
### CORRECTION
print('Après réduction ACP:')
ztrain = scaler.fit_transform(X_train)
ztest = scaler.transform(X_test)
acp = PCA(n_components=3)
Xtrain_p = acp.fit_transform(ztrain)
Xtest_p = acp.transform(ztest)
ztrain = scaler.fit_transform(X_train)
%timeit nb.fit(Xtrain_p, y_train)
y_pred = nb.predict(Xtest_p)
acc = accuracy_score(y_test, y_pred)
print('accuracy: ',acc)

4. puis celui de la LDA

In [ ]:
### CORRECTION
print('Après réduction LDA:')
lda = LinearDiscriminantAnalysis()
Xtrain_p = lda.fit_transform(X_train, y_train)
Xtest_p = lda.transform(X_test)
nb = GaussianNB()
%timeit nb.fit(Xtrain_p, y_train)
y_pred = nb.predict(Xtest_p)
acc = accuracy_score(y_test, y_pred)
print('accuracy: ',acc)